# command_led_example.py

In [ ]:
import hebi
from time import sleep

lookup = hebi.Lookup()
group = lookup.get_group_from_names(['family'], ['name'])

if group is None:
    print('Group not found: Did you forget to set the module family and names above?')
    exit(1)

# Set command lifetime to 100 ms
group.command_lifetime = 100

group_command = hebi.GroupCommand(group.size)

# Set the LED red
group_command.led.color = 'red'
group.send_command(group_command)

sleep(3)

# Set the LED Blue, and then clear the command.
# Note that this "clear" does not return the LED to module control, but
# rather remove any LED command from the command object, so when this is sent
# to the module the LED state won't be affected.
group_command.led.color = 'blue'
group_command.led.color = None
group.send_command(group_command)

sleep(3)

# Set the LED to "module control" mode; the first three digits are ignored
# if the 'alpha' value is zero.
group_command.led.color = 'transparent'
group.send_command(group_command)

sleep(3)

# Set the LED purple. Note that this override automatically sets the alpha
# channel to "255" (e.g., arguments are RGB).
group_command.led.color = hebi.Color(0, 255, 255)
group.send_command(group_command)

sleep(3)
  
# Set the LED to module control.
group_command.led.color = 'transparent'
group.send_command(group_command)


# command_position_example.py

In [ ]:
#!/usr/bin/env python3

import hebi
import numpy as np
from time import sleep
from math import sin

lookup = hebi.Lookup()
group = lookup.get_group_from_names(['X5-4'], ['X5-0000'])

if group is None:
    print('Group not found: Did you forget to set the module family and names above?')
    exit(1)

command = hebi.GroupCommand(group.size)

# Send commands to the group in a loop.
# Note that these packets may be dropped if network traffic is too high, so
# be sure to close a feedback loop at the high level!
period = 0.5
t = 0.0

positions = np.empty(group.size, dtype=np.float64)
while t < 10.0:
    for module_index in range(0, group.size):
        positions[module_index] = sin((t * 0.5) + (module_index * 0.25))

    command.position = positions
    group.send_command(command)
    sleep(period)
    t = t + period

# For critical packets, we can verify that they were sent by requesting
# confirmation from the group.  If the acknowledgement function returns
# 'True', it got positive confirmation. If it returns 'False', EITHER:
# - the sent command was dropped
# - the sent command was received by the group, but its response was either
#   dropped or the timeout period expired before receipt of the group.
# Again, a high-level process should intelligently handle these conditions!
# Note that this is a blocking call, and so for high-frequency applications,
# send_command should be used instead.
timeout_ms = 100

while t < 10.0:
    for module_index in range(0, group.size):
        positions[module_index] = sin((t * 0.5) + (module_index * 0.25))

    command.position = positions
    if group.send_command_with_acknowledgement(command):
        print('Got acknowledgement.')
    else:
        print('Did not receive acknowledgement!')

    sleep(period)
    t = t + period
